# Heuristic based compound word splitter for NLP applications



## A. Data Preparation

In this project, we will use corpus word frequency data from:
https://www.ids-mannheim.de/digspra/kl/projekte/methoden/derewo/. Refer to README for more details.

In [1]:
dereko_url = 'http://www.ids-mannheim.de/fileadmin/kl/derewo/DeReKo-2014-II-MainArchive-STT.100000.freq.zip'
dereko_filename = 'DeReKo-2014-II-MainArchive-STT.100000.freq.zip'

In [2]:
import urllib.request
urllib.request.urlretrieve(dereko_url, dereko_filename)

('DeReKo-2014-II-MainArchive-STT.100000.freq.zip',
 <http.client.HTTPMessage at 0x266415c3310>)

In [3]:
import zipfile
with zipfile.ZipFile(dereko_filename, 'r') as zip_ref:
    zip_ref.extractall('.')

In [4]:
print(open('DeReKo-2014-II-MainArchive-STT.100000.freq', encoding='utf-8').read()[:110])

,	,	$,	500367688
.	.	$.	481370234
der	die	ART	241408360.16429
die	die	ART	188943867.569055
und	und	KON	1863515


In [6]:
import csv

# Storing data in a Python list
data = []
with open('DeReKo-2014-II-MainArchive-STT.100000.freq', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        data.append(row)

# Check data entries
checked_data = []
for entry in data:
    try:
        word, lemma, pos, freq = entry
        checked_data.append(entry)
    except ValueError:
        # might happen with '\t', we can ignore this one.
        print('Error on %s, removing this entry.' % str(entry))
data = checked_data

print('Loaded %d Words + POS + Frequencies.' % len(data))

Error on ['\t', '$(', '156259594'], removing this entry.
Loaded 99999 Words + POS + Frequencies.


In [7]:
#data is in the sequence - words, stem, part-of-speech, frequency
print(data[0])
print(data[1])
print(data[2])
print(data[5])
print()
print(data[-7])
print(data[-6])
print(data[-5])
print(data[-4])

[',', ',', '$,', '500367688']
['.', '.', '$.', '481370234']
['der', 'die', 'ART', '241408360.16429']
['in', 'in', 'APPR', '140040898']

['Hohepriester', 'Hohepriester', 'NN', '3754.72208']
['Graphic', 'UNKNOWN', 'NE', '3754.71645999999']
['Klein-Winternheim', 'unknown', 'NE', '3754.5969999999']
['Londoner', 'Londoner', 'NN', '3754.53195200004']


In [8]:
#Checking top occuring POS tags from the word frequency list to limit the number of POS components in compound splitter

N_c = {}

from collections import Counter
for entry in data:
        word, lemma, pos, freq = entry
        if pos in N_c:
            N_c[pos] += 1
        else:
            N_c[pos] = 1
            
N_c_sorted = dict(sorted(N_c.items(), key=lambda item: item[1], reverse = True))

print(N_c_sorted)
print(sum(N_c_sorted.values()))
#print(N_c.most_common(20))

{'NN': 41970, 'NE': 19699, 'ADJA': 12295, 'VVFIN': 6711, 'CARD': 4463, 'ADJD': 4455, 'VVPP': 3437, 'VVINF': 2792, 'ADV': 981, 'TRUNC': 545, 'VVIZU': 490, 'FM': 373, 'APPR': 220, 'PTKVZ': 176, 'PIAT': 152, 'PIS': 126, 'XY': 121, 'VMFIN': 89, 'VVIMP': 89, 'PROAV': 84, 'PPOSAT': 74, 'VAFIN': 72, 'KOUS': 66, 'PWAV': 50, 'APPRART': 49, 'PDS': 47, 'KON': 45, 'PDAT': 44, 'PPER': 39, 'ART': 35, 'ITJ': 35, 'PRELS': 18, 'PWS': 18, 'PAV': 15, '$(': 12, 'PTKANT': 12, 'APPO': 12, 'PRF': 11, 'KOUI': 9, 'PWAT': 9, '$.': 6, 'PTKA': 6, 'APZR': 6, 'VAIMP': 6, 'KOKOM': 5, 'VAINF': 5, 'VMINF': 5, 'VAPP': 4, 'PPOSS': 4, 'PTKNEG': 3, 'PTKZU': 3, 'VMPP': 3, 'PRELAT': 2, '$,': 1}
99999


# B. Compound splitting tools

In [9]:
from abc import ABC, abstractmethod

class CompoundSplitter(ABC):

    def __init__(self):
        pass

    @abstractmethod
    def csplit(self, word):
        pass

In [10]:
## Some baseline splitters for comparison of results

class BaselineSplitter1(CompoundSplitter):
    # does not split anything

    def csplit(self, word):
        return word


class BaselineSplitter2(CompoundSplitter):
    # Splits if word starts with another word of 4-7 characters.
    # Why 4-7? Too short words might not be useful here, too long words might be also compounds.

    def __init__(self):
        self.components = []
        self._init_components()

    def _init_components(self):
        # load word list from frequency data
        for entry in data:
            word, lemma, pos, freq = entry
            if 4 <= len(word) <= 7:
                self.components.append(word)

    def csplit(self, word):
        for component in self.components:
            if word[:len(component)] == component:
                return component + '|' + word[len(component):]
        return word

baseline_1 = BaselineSplitter1()
baseline_2 = BaselineSplitter2()

In [11]:
# This is the main splitter class which contains methods for generating word splits
# As an exercise, you can also implement your own splitter here

class MySplitter(CompoundSplitter):

    import time 
    
    def __init__(self):
        self.complete_list = []
        self.components = []
        self.frequencies = []
        self.most_frequent = []
        self.verbs = []
        
        #These are verb POS tags -- can be excluded from corpus
        self.verbPOS = ['VAFIN','VAIMP','VAINF','VAPP','VMFIN','VMINF','VMPP','VVFIN','VVIMP','VVINF','VVIZU','VVPP']
        
        #These suffices can be present with lemmas to produce various inflected forms
        self.JoinWords = ['s','es','n','er','e','en']
        self._init_components()

    def _init_components(self):
        # load corresponding word list from frequency data
        for entry in data:
            word, lemma, pos, freq = entry
            self.complete_list.append(word.encode('utf-8').lower())
            self.frequencies.append([word.encode('utf-8').lower(), freq])
            if float(freq) >= 4000000:
                self.most_frequent.append(word.encode('utf-8').lower())
            if pos in self.verbPOS:
                self.verbs.append(word.encode('utf-8').lower())
            if 4 <= len(word) <= 11:
                self.components.append([word, lemma, freq])
    
    
    def match_comp(self, word, component):
        if word.encode('utf-8').lower() == component.encode('utf-8').lower():
            return word
        return ''
    
    
    def list_diff(self, l1, l2):
        diff =  list(set(l1) - set(l2)) + list(set(l2) - set(l1))
        return str('|'.join(diff))
        
        
    def split_heads(self, word):
        '''Generates head splits for a given word -- first possible split reading word from left to right'''
        head_splits = []
        for entry in self.components:
            comp, lemma, freq = entry
            for end in (self.JoinWords + ['']):
                component = comp+end
                head = word[:len(component)]
                match = self.match_comp(head, component) 
                if match:
                    if match not in self.JoinWords:
                        string = match + '|' + word[len(match):]
                        if string not in head_splits:
                            if string[-1] != '|':
                                head_splits.append(string)
        return head_splits
   

    def split_tails(self, word):
        '''Generates tail splits for a given word -- first possible split reading word reverse from right to left'''
        tail_splits = []
        for entry in self.components:
            component, lemma, freq = entry
            tail = word[len(word) - len(component):]
            match = self.match_comp(tail, component) 
            if match:
                if match not in self.JoinWords:
                    i = len(word)-len(match)
                    if word[:i]:
                        string = word[:i] + '|' + match
                        if string not in tail_splits:
                            tail_splits.append(string)
        return tail_splits
    
    
    def csplit(self, word):
        '''Generates best possible split for a compound word -- main working function'''
        import time
        start = time.perf_counter()
        sub_splits = []
        self.base_splits = self.split_heads(word)
        base_tails = self.split_tails(word)        
        for split in base_tails:
            if split not in self.base_splits:
                self.base_splits.append(split)   
        self.all_splits = self.base_splits
        
        for s in self.base_splits:
            words = s.split('|')
            for i in range(len(words)):
                if len(words[i]) > 9:
                    rem_words = words[:i] + words[i+1:]
                    rem_heads = self.split_heads(words[i])
                    rem_tails = self.split_tails(words[i])
                    rem_splits = rem_heads + rem_tails
                    for rem in rem_splits:
                        if rem.split('|')[0] not in self.JoinWords:
                            if i==0:
                                string = rem + '|'.join(rem_words)
                            if i==1:
                                string = '|'.join(rem_words) + '|' + rem
                            if string[-1] != '|' and string.replace('|','') == word:
                                if string not in self.all_splits:
                                    self.all_splits.append(string)       
        
        self.all_splits.append(word)
        sp1 = time.perf_counter()
        #print('Evaluation starts now. Time till now - in s: ', round(sp1 - start,3))
    
        self.evaluated_splits = dict()
        for split in self.all_splits:
            val = self.evaluate_split(split)
            self.evaluated_splits[split] = val
        self.sorted_evals =  dict(sorted(self.evaluated_splits.items(), key=lambda item: item[1], reverse = True)) 
        
        sp2 = time.perf_counter()
        #print('Evaluation time - in s: ', round(sp2 - sp1,3))
        if self.sorted_evals:
            return next(iter(self.sorted_evals))
        else:
            return word
        
        
    def evaluate_split(self, split_to_eval):
        '''Evaluate a split and assign score value to it'''
        val = 1
        words = split_to_eval.split('|')
        for word in words:
            freq = self.get_freq(word)
            if len(word) <= 2:
                val *= len(word)
            elif len(word) == 3:
                val *= 0.1*freq*len(word)
            else:
                val *= freq*len(word)
        val = val**(1/len(words))
        return [val, [len(word) for word in words]]
    
    
    def get_freq(self, word):
        '''Get frequency of closest lemma match'''
        for entry in self.frequencies:
            wd, freq = entry
            if wd == word.encode('utf-8').lower():
                    return float(freq)
            else:
                for end in self.JoinWords:
                    end = end.encode('utf-8')
                    if (wd+end) == word.encode('utf-8').lower():
                        return float(freq)
                    if (end+wd) == word.encode('utf-8').lower():
                        return float(freq)
        return 0.5
    
    
    def csplit_head(self, word):
        '''Used only if head side splits are required'''
        pred = self.csplit(word)
        for split in self.all_splits:
            if split.count('|') > 1:
                del self.sorted_evals[split]
        if self.sorted_evals:
            return next(iter(self.sorted_evals))
        else:
            return word
          
    
    def csplit_test(self, word):
        '''Can be used for debugging'''
        pred = self.csplit(word)
        #print(self.base_splits)
        print(self.sorted_evals)
        return pred
    
    
    def csplit_test_head(self, word):
        pred = self.csplit_head(word)
        print(self.sorted_evals)
        return pred
    
my_splitter = MySplitter()

In [12]:
print(my_splitter.csplit('Stammesorganisation'))
#print(my_splitter.csplit('Ausbildungsende'))
#print(my_splitter.csplit('Oberstufenunterricht'))

Stammes|organisation


In [13]:
print(my_splitter.csplit('Telephongespräch'))
print(my_splitter.csplit('Wasserschüssel'))

Telephon|gespräch
Wasser|schüssel


In [14]:
print(my_splitter.csplit('Stuttgarter'))
print(my_splitter.csplit('Busfahrerin'))

Stuttgarter
Bus|fahrerin


# C. Evaluating results quality

Download the file `csplits_train.tsv` and `csplits_test.tsv`.

This data set was created from a mixture of German social media data from Twitter and Reddit. This data set is created automatically, there might be errors (tokenization/POS/compound splits). The words were tokenized, tagged and lemmatized using TreeTagger.

While `csplits_train.tsv` contains lemmata (extracted from the TreeTagger output) with the predicted splits of a compound splitter (we used CompoST (Cap, 2014) based on SMOR; you should not use these tools, you should develop your own!), `NLP2021_csplits_test.tsv` only contains lemmata. These lists contain **only nouns** (NN according to TreeTagger) and each word has at least 10 characters ([A-Za-zÄÜÖöüäß], and only those are allowed - so no numbers, etc.).

The TreeTagger is a tool for annotating text with part-of-speech and lemma information. It was developed by Helmut Schmid in the TC project at the Institute for Computational Linguistics of the University of Stuttgart. Refer https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/ for more details

In [15]:
train_data_file = 'csplits_train.tsv'
train_data_instances = [instance.split('\t') for instance in
                        open(train_data_file).read().split('\n')
                        if instance.strip()]
print('Read %d training data instances.' % len(train_data_instances))

test_data_file = 'csplits_test.tsv'
test_data_instances = open(test_data_file).read().split('\n')[:-1]
print('Read %d test data instances.' % len(test_data_instances))

Read 20000 training data instances.
Read 8345 test data instances.


In [17]:
print('** Training data with predicted splits:')
print(train_data_instances[1])
print(train_data_instances[2])
print(train_data_instances[0])
print(train_data_instances[51])

print('\n** Test data:')
print(test_data_instances[0])
print(test_data_instances[10])
print(test_data_instances[250])

** Training data with predicted splits:
['Grundstandard', 'Grund|standard']
['Stammesorganisation', 'Stammes|organisation']
['Heilerziehungspflegerin', 'Heil|erziehungs|pflegerin']
['Verbesserung', 'Verbesserung']

** Test data:
Beschleunigungsrennen
Paketinhalt
Lagerverwaltung


Splits are marked with vertical bars ('|'). Note that some words are split multiple times (>1 bars) while some words are not split at all (no bars) - some words here are simply long nouns (e.g. derived nouns) and not compounds. Also note that the components of these words can be combined with joint elements ("Fugenelemente").
For example, `Verkehrsinformationssystem` is a compound of `Verkehr`, `Information` and `System` with two additional `s` inserted: `Verkehr**s**|information**s**|system`.

Also note that verb components often occur in stem form: 
`Denkvermögen` is a compound of the verb `denken` and the noun `vermögen`; the stem of `denken` is `denk`, thus, we get `Denk|vermögen`.

Joint elements can also occur with verb stems:
`Funkmeldesystem` is a compound of `Funk`, `melden` and `System`. `meld` is the stem of of `melden`. Thus the additional `e` is a joint element in the split form `Funk|meld**e**|system` .


## C.1 Evaluation on train data




In [41]:
#Defining some function that calls the above splitter methods and compare with gold standard labels

def evaluate(compound_splitter, labeled_instances):
    tp = 0
    for labeled_instance in labeled_instances:
        word, split = labeled_instance
        pred = compound_splitter.csplit(word)
        if pred == split:
            tp += 1
    acc = tp/len(labeled_instances)
    return acc


#Smaller version of evaluate method to run preliminary tests on a part of the trained dataset - first n entries
def evaluate_2(compound_splitter, labeled_instances, n):
    tp = 0
    for labeled_instance in labeled_instances[:n]:
        word, split = labeled_instance
        pred = compound_splitter.csplit(word)
        if pred == split:
            tp += 1
    acc = tp/len(labeled_instances[:n])
    return acc
    
    
#Function to evaluate first n entries and save results in a file to compare results
def evaluate_and_save(compound_splitter, labeled_instances, n, filename):
    with open(filename, mode='w') as outfile:
        outfile.truncate()
        tp = 0
        i = 0
        for labeled_instance in labeled_instances[:n]:
            i += 1
            if i%(n/10) == 0:
                print('Completed ', i, 'instances')
            word, split = labeled_instance
            pred = compound_splitter.csplit(word)
            if pred == split:
                tp += 1
                outfile.write('Correct -  ' + word + '\t\t Predicted:' + pred + '\t\t Correct:' + split + '\n')
            else:
                outfile.write('Incorrect -  ' + word + '\t\t Predicted:' + pred + '\t\t Correct:' + split + '\n')
    outfile.close()
    acc = tp/len(labeled_instances[:n])
    return acc

In [42]:
baseline_1 = BaselineSplitter1()
print('Accuracy of baseline_1: %2.2f%%' % (100*evaluate_2(baseline_1, train_data_instances, 100)))

baseline_2 = BaselineSplitter2()
print('Accuracy of baseline_2: %2.2f%%' % (100*evaluate_2(baseline_2, train_data_instances, 100)))

Accuracy of baseline_1: 12.00%
Accuracy of baseline_2: 28.00%


In [52]:
#Evaluate first n instances and save results in txt file for comparison
import time
filename = 'Test_Eval_first50.txt'
n = 100

start = time.perf_counter()
print('Accuracy of my splitter - on first n instances: %2.2f%%' % (100*evaluate_and_save(my_splitter, train_data_instances, n, filename)))
end = time.perf_counter()
print('Time taken to evaluate and save n instances in secs is: ', round(end-start, 2))

Completed  5 instances
Completed  10 instances
Completed  15 instances
Completed  20 instances
Completed  25 instances
Completed  30 instances
Completed  35 instances
Completed  40 instances
Completed  45 instances
Completed  50 instances
Accuracy of my splitter - on first n instances: 72.00%
Time taken to evaluate and save n instances in secs is:  220.03


In [58]:
#Bankaktivität
word = 'Bankaktivität'
print(my_splitter.csplit_test(word))

{'Bank|aktivität': [235250.6766834051, [4, 9]], 'Bankaktivität': [0.5, [13]]}
Bank|aktivität


## C.2 Prediction of test data

In [12]:
def predict_and_save(compound_splitter, instances, filename):
    with open(filename, mode='w') as outfile:
        for word in instances:
            pred = compound_splitter.csplit(word)
            outfile.write(word + '\t' + pred + '\n')

            
#Final evaluation of all words and saving data in a txt file
def predict_and_save_final(compound_splitter, instances, filename):
    with open(filename, mode='w') as outfile:
        outfile.truncate()
        i = 0
        for word in instances[:]:
            i += 1
            if i%500 == 0:
                print('Completed ', i, 'instances')
            pred = compound_splitter.csplit(word)
            outfile.write(word + '\t' + pred + '\n')
    outfile.close()

In [355]:
# Predicting test data with simple baselines
baseline_1 = BaselineSplitter1()
predict_and_save(baseline_1, test_data_instances, 'pred-components_baseline_1.csv')

baseline_2 = BaselineSplitter2()
predict_and_save(baseline_2, test_data_instances, 'pred-components_baseline_2.csv')

In [13]:
# Predicting test data using our own splitter here -- saving results in csv file
filename = 'pred-components.csv'

my_splitter = MySplitter()
predict_and_save_final(my_splitter, test_data_instances, filename)

Completed  500 instances
Completed  1000 instances
Completed  1500 instances
Completed  2000 instances
Completed  2500 instances
Completed  3000 instances
Completed  3500 instances
Completed  4000 instances
